In [20]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
from crewai import Agent,LLM,Crew,Task
from langchain_community.utilities import GoogleSerperAPIWrapper
from crewai.tools import tool

In [22]:
llm = LLM(model="groq/llama-3.3-70b-versatile", temperature=0.2)

In [ ]:
@tool()
def google_search(query: str) -> str:
    """Perform a Google web search and return the results."""
    search = GoogleSerperAPIWrapper()
    return search.run(query)

In [ ]:
# markdown example

## markdown

The markdown parameter enables automatic markdown formatting for task outputs. When set to True, the task will instruct the agent to format the final answer using proper Markdown syntax.

When markdown=True, the agent will receive additional instructions to format the output using:

# for headers

**text** for bold text

*text* for italic text

- or * for bullet points

`code` for inline code

language ``` for code blocks

In [ ]:
'''Benefits of Markdown Output

Consistent Formatting: Ensures all outputs follow proper markdown conventions
Better Readability: Structured content with headers, lists, and emphasis
Documentation Ready: Output can be directly used in documentation systems
Cross-Platform Compatibility: Markdown is universally supported'''

In [ ]:
# Create Research Agent
researcher = Agent(
    role='Technology Researcher with specialist in researching recent news',
    goal='Research and gather comprehensive information about emerging technologies',
    backstory="""You are an expert technology researcher with a keen eye for 
    identifying trends and innovations. You excel at finding credible sources 
    and synthesizing complex information into clear insights.""",
    verbose=True, 
    allow_delegation=False,
    llm=llm
)

# Create Research Task
research_task = Task(
    description="""
    Topic {topic}. 
    using the given topic to perform the research. 
    Focus on:
    1. Key features and capabilities
    2. Popular frameworks in 2025
    3. Use cases and applications
    Provide a detailed summary.""",
    agent=researcher,
    markdown = True,  # Enable markdown formatting for the final output
    expected_output="A comprehensive research report on AI agent frameworks stict 500 tokens"
)

# Create Crew
crew = Crew(
    agents=[researcher],
    tasks=[research_task],
    verbose=True
)


In [ ]:
result = crew.kickoff(inputs={"topic": "AI agent frameworks"})

In [ ]:
print("Final Result:")
print(result)

# multi agent orchestration example

In [ ]:
research_agent = Agent(
    role="Senior Research Analyst",
    goal="Search the internet and gather accurate, up-to-date information about the given topic.",
    backstory=(
        "You are an experienced research analyst who specializes in gathering "
        "reliable and structured information from the web. "
        "You always verify sources and summarize clearly."
    ),
    llm = llm,
    tools=[google_search],
    verbose=True,
    max_tokens=500
)

#Keyword Extraction Agent
keyword_agent = Agent(
    role="SEO Keyword Strategist",
    goal="Extract important keywords, phrases, and semantic terms from research data.",
    backstory=(
        "You are an SEO specialist with 10+ years of experience in content optimization. "
        "You know how to identify high-impact keywords and trending phrases "
        "that improve blog ranking."
    ),
    llm = llm,
    verbose=True,
    max_tokens=500
)

# Blog Writer Agent
blog_writer_agent = Agent(
    role="Professional Blog Writer",
    goal="Write an engaging, SEO-optimized blog using provided keywords and research.",
    backstory=(
        "You are a creative content writer who writes high-quality, "
        "engaging, and reader-friendly blog posts. "
        "You naturally integrate keywords without keyword stuffing."
    ),
    llm = llm,
    verbose=True,
    max_tokens=500
)


In [ ]:
'''SEO (Search Engine Optimization) is the process of improving your website or content so that it appears higher in search engine results like Google.'''

In [12]:
# Task 1: Research
research_task = Task(
    description="""
    Conduct detailed web research about: {topic}.
    Summarize key insights, statistics, and trends.""",
    expected_output="A detailed research summary with facts and insights.",
    agent=research_agent,
)

# Task 2: Extract Keywords
keyword_task = Task(
    description="From the research summary, extract 15-20 high-value SEO keywords "
                "and important phrases relevant to the topic.",
    expected_output="A structured list of keywords and phrases.",
    agent=keyword_agent
)

# Task 3: Write Blog
blog_task = Task(
    description="Using the research summary and extracted keywords, "
                "write a 500-word SEO-optimized blog article. "
                "Include introduction, subheadings, and conclusion.",
    expected_output="A well-structured blog article in markdown format.",
    agent=blog_writer_agent
)


In [13]:
crew = Crew(
    agents=[research_agent, keyword_agent, blog_writer_agent],
    tasks=[research_task, keyword_task, blog_task],
    verbose=True
)



In [ ]:
result = crew.kickoff()
print(result)